#**This file contains the models for all four cases (two each for TASK A and TASK B).**

Including Relevent Library files.

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import time
import pickle as cp
from sliding_window import sliding_window

Using TensorFlow backend.


Loading Opportunity Dataset and including accessory files.

In [0]:
#Loading Opportunity Dataset.
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip --no-check-certificate

--2020-05-23 11:07:46--  https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306636009 (292M) [application/x-httpd-php]
Saving to: ‘OpportunityUCIDataset.zip.1’

OpportunityUCIDatas 100%[===================>] 292.43M  20.8MB/s    in 15s     

2020-05-23 11:08:02 (19.5 MB/s) - ‘OpportunityUCIDataset.zip.1’ saved [306636009/306636009]



In [6]:
#Including preprocessing file.
!python preprocess_data.py -h

usage: preprocess_data.py [-h] -i INPUT -o OUTPUT [-t {gestures,locomotion}]

Preprocess OPPORTUNITY dataset

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        OPPORTUNITY zip file
  -o OUTPUT, --output OUTPUT
                        Processed data file
  -t {gestures,locomotion}, --task {gestures,locomotion}
                        Type of activities to be recognized


# TASK A:- Applying Model on Locomotion Dataset

**TASK A:- Applying Model on Locomotion Dataset (with NULL class)**

Preprocessing the data from data set.

In [36]:
#Selecting the data relevent to TASK A (Detecting Locomotion).
!python preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data -t locomotion

Checking dataset data/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat
... file OpportunityUCIDataset/dataset/S3-Drill.dat
... file OpportunityUCIDataset/dataset/S3-ADL1.dat
... file OpportunityUCIDataset/dataset/S3-ADL2.dat
... file OpportunityUCIDataset/dataset/S3-ADL3.dat
... file OpportunityUCIDataset/dataset/S2-ADL4.dat
... file OpportunityUCIDataset/dataset/S2-ADL5.dat
... file OpportunityUCIDataset/dataset/S3-ADL4.dat
... file OpportunityUCIDataset/dataset/S3-ADL5.dat
Fi

In [0]:
# Number of Sensor Channels used in the OPPORTUNITY dataset.
NB_SENSOR_CHANNELS = 113

# Number of classes in which data is classified (or to be classified).
NUM_CLASSES = 5

# Length of the sliding window used to segmenting the time-series-data.
SLIDING_WINDOW_LENGTH = 24

# Length of the input sequence after convolutional operations
FINAL_SEQUENCE_LENGTH = 8

# Steps of the sliding window used in segmenting the data.
SLIDING_WINDOW_STEP = 12

# Variable for Batch Size.
BATCH_SIZE = 100

# Number filters used in convolutional layers.
NUM_FILTERS = 64

# Size of filters used in convolutional layers.
FILTER_SIZE = 5

# Units in the long short-term recurrent layers.
NUM_UNITS_LSTM = 128

Loading the data and segmenting it according to length of sliding window.

In [38]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading Data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))

Loading Data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)


In [39]:
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
X_train = X_train.reshape((-1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))
X_train.shape

 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)


(46495, 24, 113, 1)

In [40]:
X_train.shape

(46495, 24, 113, 1)

Defining DeepConvLSTM Model. 

In [41]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential()
model.add(keras.Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1)))

#intializing randomly orthogonal weights.
initializer = tf.keras.initializers.Orthogonal()

#Adding 4 layes of CNN.
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Permute((2,1,3)))
model.add(layers.Reshape( (int(model.layers[4].output_shape[1]), int(model.layers[4].output_shape[2]) * int(model.layers[4].output_shape[3]))))

#Adding 2 layers of LSTM.
model.add(layers.LSTM(NUM_UNITS_LSTM,dropout=0.5,return_sequences=True,kernel_initializer=initializer))
model.add(layers.LSTM(NUM_UNITS_LSTM,dropout=0.5,return_sequences=True,kernel_initializer=initializer))
model.add(layers.Flatten())

#Applying dense layer of Softmax to the output of 4-CNN layers and 2 LSTM layers.
model.add(layers.Dense(NUM_CLASSES, activation="softmax"))
    
#Printing Summery of Model.
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 20, 113, 64)       384       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 113, 64)       20544     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 12, 113, 64)       20544     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 8, 113, 64)        20544     
_________________________________________________________________
permute_2 (Permute)          (None, 113, 8, 64)        0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 113, 512)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 113, 128)         

In [42]:
model.layers[4].output_shape


(None, 113, 8, 64)

Encoding the training and testing data to One-Hot Encoded form.

In [0]:
from sklearn.preprocessing import OneHotEncoder
def prepare_targets(y_train, y_test):
	ohe = OneHotEncoder()
	ohe.fit(y_train)
	y_train_enc = ohe.transform(y_train)
	y_test_enc = ohe.transform(y_test)
	return y_train_enc.A, y_test_enc.A

In [0]:
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

Compiling Model

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [46]:
y_train.shape

(46495, 1)

Training Model

In [0]:
#model.fit(X_train, y_train_enc, batch_size=BATCH_SIZE, epochs=50,validation_split=0.1)

#The model is already trained using above statement and trained model is loaded for testing data.

Load Model.

In [0]:
from tensorflow.keras.models import load_model

new_model1 = load_model('task_a_with_null_working.h5')

Testing our Model on test data.

In [0]:
y_prob = new_model1.predict(X_test) 
y_classes = y_prob.argmax(axis=-1)

Printing F1 Score and relevent Result Metrics.

In [50]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

precision, recall, f1_score, supp = precision_recall_fscore_support(y_classes, y_test, average='micro')


print(confusion_matrix(y_classes, y_test))
print("PRECISION: {0}" .format(precision))
print("RECALL: {0}".format(recall))
print("F1_SCORE : {0}".format(f1_score))

[[1531   20  107   18   11]
 [ 226 2917  441   25    1]
 [ 252  134 1719    0    0]
 [  28   30    5 1969   16]
 [   5    0    0    4  435]]
PRECISION: 0.8662825955124318
RECALL: 0.8662825955124318
F1_SCORE : 0.8662825955124317


**Task A:- Applying Model on Locomotion Dataset (Without NULL class)**

Preprocessing the data from data set.

In [51]:
!python preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data -t locomotion

Checking dataset data/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat
... file OpportunityUCIDataset/dataset/S3-Drill.dat
... file OpportunityUCIDataset/dataset/S3-ADL1.dat
... file OpportunityUCIDataset/dataset/S3-ADL2.dat
... file OpportunityUCIDataset/dataset/S3-ADL3.dat
... file OpportunityUCIDataset/dataset/S2-ADL4.dat
... file OpportunityUCIDataset/dataset/S2-ADL5.dat
... file OpportunityUCIDataset/dataset/S3-ADL4.dat
... file OpportunityUCIDataset/dataset/S3-ADL5.dat
Fi

Defining important variables.

In [0]:
# Variable for Number of sensor channels in the OPPORTUNITY dataset.
NB_SENSOR_CHANNELS = 113

# Variable for number of classes in which data is classified (or to be classified). 
NUM_CLASSES = 4

# Variable for length of the sliding window used in segmenting the time-series-data.
SLIDING_WINDOW_LENGTH = 24

# Length of the input sequence after convolutional operations
FINAL_SEQUENCE_LENGTH = 8

# Variable for steps of the sliding window used in segmenting the data
SLIDING_WINDOW_STEP = 12

# Variable for Batch Size
BATCH_SIZE = 100

# Number filters used in convolutional layers
NUM_FILTERS = 64

# Size of each filter convolutional layers
FILTER_SIZE = 5

# Number of unit in the LSTM layer.
NUM_UNITS_LSTM = 128

Loading Data an segmenting data according to sliding window length.

In [53]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))

Loading data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)


In [54]:
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
X_train = X_train.reshape((-1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))
print(X_train.shape)
print(y_train.shape)

 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)
(46495, 24, 113, 1)
(46495,)


Defining Model.

In [55]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential()
model.add(keras.Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1)))

# intializing randomly orthogonal weights.
initializer = tf.keras.initializers.Orthogonal()

# Adding 4 layers of CNN
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu", kernel_initializer = initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer = initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer = initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer = initializer))
model.add(layers.Permute((2,1,3)))
model.add(layers.Reshape( (int(model.layers[4].output_shape[1]), int(model.layers[4].output_shape[2]) * int(model.layers[4].output_shape[3]))))

# Adding 2 layers of LSTM.
model.add(layers.LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model.add(layers.LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model.add(layers.Flatten())

# Adding a dense layer of softmax.
model.add(layers.Dense(NUM_CLASSES, activation="softmax"))

# Printing Model Summary.
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 20, 113, 64)       384       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 16, 113, 64)       20544     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 12, 113, 64)       20544     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 8, 113, 64)        20544     
_________________________________________________________________
permute_3 (Permute)          (None, 113, 8, 64)        0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 113, 512)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 113, 128)         

Encoding the training and testing data to One-Hot Encoded form.

In [0]:
from sklearn.preprocessing import OneHotEncoder
def prepare_targets(y_train, y_test):
	ohe = OneHotEncoder()
	ohe.fit(y_train)
	y_train_enc = ohe.transform(y_train)
	y_test_enc = ohe.transform(y_test)
	return y_train_enc.A, y_test_enc.A

Removing NULL class values.

In [57]:
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

count = 0
idx = []
for i in range(0,y_train.shape[0]):
  if(y_train[i] == 0):
    count += 1
    idx.append(i)
print(len(idx))
y_train_new = np.delete(y_train,idx)
y_train_new = y_train_new.reshape(-1,1)
print(y_train_new.shape)

print(X_train.shape)
X_train_new = np.delete(X_train,idx,axis=0)
print(X_train_new.shape)

count = 0
idx = []
for i in range(0,y_test.shape[0]):
  if(y_test[i] == 0):
    count += 1
    idx.append(i)
print(len(idx))
y_test_new = np.delete(y_test,idx,axis = 0)
print(y_test_new.shape)

X_test_new = np.delete(X_test,idx,axis = 0)
print(X_test_new.shape)

y_train_enc, y_test_enc = prepare_targets(y_train_new, y_test_new)
print(y_train_enc.shape)
print(y_test_enc.shape)

7680
(38815, 1)
(46495, 24, 113, 1)
(38815, 24, 113, 1)
2042
(7852, 1)
(7852, 24, 113, 1)
(38815, 4)
(7852, 4)


In [58]:
y_train_enc

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

Compiling Model

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [60]:
X_train.shape

(46495, 24, 113, 1)

Training Model

In [0]:
#model.fit(X_train_new, y_train_enc, batch_size=BATCH_SIZE, epochs=50,validation_split=0.1)

#Model is already trained using above lines of code and loaded directly for testing purposes.

Loading Model.

In [0]:
from tensorflow.keras.models import load_model

new_model2 = load_model('task_a_without_null_working.h5')

Testing our Model on test data.

In [63]:
y_prob = new_model2.predict(X_test_new) 
#print(y_prob)

#adjusting format of output to the required format of result.
y_classes = y_prob.argmax(axis=-1)

for i in range(0,y_classes.shape[0]):
  y_classes[i] += 1
print(np.sum(y_classes))
print(np.sum(y_test_new))

15400
15545


In [64]:
y_prob.shape

(7852, 4)

Printing F1 Score and relevent Result Metrics.

In [65]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1_score, supp = precision_recall_fscore_support(y_test_new,y_classes,average='weighted')

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_classes, y_test_new))
#print(metrics)
print("PRECISION: {0}" .format(precision))
print("RECALL: {0}".format(recall))
print("F1_SCORE : {0}".format(f1_score))
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html

[[2853  322   43    1]
 [ 223 1948    5    3]
 [  25    2 1963    3]
 [   0    0    5  456]]
PRECISION: 0.919917164493682
RECALL: 0.9195109526235354
F1_SCORE : 0.9194705489636714


**----------------------------------------------------------------------------------------------------------------------------------------------------------------**

# TASK B:- Applying Model on Gestures Dataset.

**TASK B :- Applying Model on Gestures (with NULL Class)**

Preprocessing the data from data set.

In [7]:
!python preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data -t gestures

Checking dataset data/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat
... file OpportunityUCIDataset/dataset/S3-Drill.dat
... file OpportunityUCIDataset/dataset/S3-ADL1.dat
... file OpportunityUCIDataset/dataset/S3-ADL2.dat
... file OpportunityUCIDataset/dataset/S3-ADL3.dat
... file OpportunityUCIDataset/dataset/S2-ADL4.dat
... file OpportunityUCIDataset/dataset/S2-ADL5.dat
... file OpportunityUCIDataset/dataset/S3-ADL4.dat
... file OpportunityUCIDataset/dataset/S3-ADL5.dat
Fi

In [0]:
# Hardcoded number of sensor channels employed in the OPPORTUNITY challenge
NB_SENSOR_CHANNELS = 113

# Hardcoded number of classes in the gesture recognition problem
NUM_CLASSES = 18

# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 24

# Length of the input sequence after convolutional operations
FINAL_SEQUENCE_LENGTH = 8

# Hardcoded step of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_STEP = 12

# Batch Size

BATCH_SIZE = 100

# Number filters convolutional layers
NUM_FILTERS = 64

# Size filters convolutional layers
FILTER_SIZE = 5

# Number of unit in the long short-term recurrent layers
NUM_UNITS_LSTM = 128

Loading the data and segmenting it according to length of sliding window.

In [9]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))

Loading data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)


In [10]:
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
X_train = X_train.reshape((-1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))
X_train.shape

 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)


(46495, 24, 113, 1)

In [11]:
X_train.shape

(46495, 24, 113, 1)

Defining DeepConvLSTM Model. 

In [12]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential()
model.add(keras.Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1)))
initializer = tf.keras.initializers.Orthogonal()
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Permute((2,1,3)))
model.add(layers.Reshape( (int(model.layers[4].output_shape[1]), int(model.layers[4].output_shape[2]) * int(model.layers[4].output_shape[3]))))
model.add(layers.LSTM(NUM_UNITS_LSTM,return_sequences=True))
model.add(layers.LSTM(NUM_UNITS_LSTM,return_sequences=True))
#model.add(layers.Reshape( (-1, NUM_UNITS_LSTM)))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASSES, activation="softmax"))

    
#Printing Model Summary.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 20, 113, 64)       384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 113, 64)       20544     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 113, 64)       20544     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 113, 64)        20544     
_________________________________________________________________
permute (Permute)            (None, 113, 8, 64)        0         
_________________________________________________________________
reshape (Reshape)            (None, 113, 512)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 113, 128)          3

In [13]:
model.layers[4].output_shape

(None, 113, 8, 64)

Encoding the training and testing data to One-Hot Encoded form.

In [0]:
from sklearn.preprocessing import OneHotEncoder
def prepare_targets(y_train, y_test):
	ohe = OneHotEncoder()
	ohe.fit(y_train)
	y_train_enc = ohe.transform(y_train)
	y_test_enc = ohe.transform(y_test)
	return y_train_enc.A, y_test_enc.A

In [0]:
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

Compiling Model

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=[tf.keras.metrics.CategoricalAccuracy()])

Training Model

In [0]:
#model.fit(X_train, y_train_enc, batch_size=BATCH_SIZE, epochs=50,validation_split=0.1)

#Model is already trained and loaded directly for testing data.

In [18]:
y_train.shape

(46495, 1)

In [0]:
from tensorflow.keras.models import load_model

new_model3 = load_model('task_b_with_null_working.h5')

Testing our Model on test data.

In [0]:
y_prob = new_model3.predict(X_test) 
y_classes = y_prob.argmax(axis=-1)

In [22]:
print(y_classes.shape)

(9894,)


Printing F1 Score and relevent Result Metrics.

In [0]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1_score, supp = precision_recall_fscore_support(y_classes, y_test, average='weighted')

from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_classes, y_test))
#print(metrics)
#print("PRECISION: {0}" .format(precision))
#print("RECALL: {0}".format(recall))
print("F1_SCORE : {0}".format(f1_score))


[[7918   11   16    6   11  112   40   40   44   11   29   21    6   16
    12   64  169   34]
 [  11   28    0    8    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   9    0   54    0    3    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [  26   19    2   46    0    0    0    0    0    0    0    0    0    0
     0    0    1    0]
 [   4    0   23    0   69    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [  18    0    0    0    0  105    9    2    0    0    0    0    0    0
     0    0    0    0]
 [  13    0    0    0    0    5  104    0    0    0    0    0    0    0
     1    0    0    0]
 [  21    0    0    0    0    5    0   49    8    0    0    1    0    0
     0    0    3    0]
 [  17    0    0    0    0    0    3    3   22    0    0    0    0    0
     0    0    1    0]
 [   8    0    0    0    0    0    1    1    0   21    3    7    6    2
     0    0    0   14]
 [   6    0    0    0    0    0    3    0    0    

--------------------------------------------------------------------------------

**TASK B:- Applying Model on Gesture dataset (without NULL class)**

Preprocessing the data from data set.

In [23]:
!python preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data -t gestures

Checking dataset data/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat
... file OpportunityUCIDataset/dataset/S3-Drill.dat
... file OpportunityUCIDataset/dataset/S3-ADL1.dat
... file OpportunityUCIDataset/dataset/S3-ADL2.dat
... file OpportunityUCIDataset/dataset/S3-ADL3.dat
... file OpportunityUCIDataset/dataset/S2-ADL4.dat
... file OpportunityUCIDataset/dataset/S2-ADL5.dat
... file OpportunityUCIDataset/dataset/S3-ADL4.dat
... file OpportunityUCIDataset/dataset/S3-ADL5.dat
Fi

Defining important variables.

In [0]:
# Hardcoded number of sensor channels employed in the OPPORTUNITY challenge
NB_SENSOR_CHANNELS = 113

# Hardcoded number of classes in the gesture recognition problem
NUM_CLASSES = 17

# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 24

# Length of the input sequence after convolutional operations
FINAL_SEQUENCE_LENGTH = 8

# Hardcoded step of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_STEP = 12

# Batch Size

BATCH_SIZE = 100

# Number filters convolutional layers
NUM_FILTERS = 64

# Size filters convolutional layers
FILTER_SIZE = 5

# Number of unit in the long short-term recurrent layers
NUM_UNITS_LSTM = 128

Loading Data an segmenting data according to sliding window length.

In [25]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))

Loading data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)


In [26]:
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
X_train = X_train.reshape((-1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))
print(X_train.shape)
print(y_train.shape)

 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)
(46495, 24, 113, 1)
(46495,)


Defining DeepConvLSTM Model.

In [27]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential()
model.add(keras.Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1)))

#intializing weights
initializer = tf.keras.initializers.Orthogonal()

#Adding 4 CNN layers.
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu", kernel_initializer = initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer = initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer = initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer = initializer))
model.add(layers.Permute((2,1,3)))
model.add(layers.Reshape( (int(model.layers[4].output_shape[1]), int(model.layers[4].output_shape[2]) * int(model.layers[4].output_shape[3]))))

#Adding 2 LSTM layers.
model.add(layers.LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model.add(layers.LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
#model.add(layers.Reshape( (-1, NUM_UNITS_LSTM)))
model.add(layers.Flatten())

#Applying a dense layer of softmax.
model.add(layers.Dense(NUM_CLASSES, activation="softmax"))


model.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 20, 113, 64)       384       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 113, 64)       20544     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 113, 64)       20544     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 113, 64)        20544     
_________________________________________________________________
permute_1 (Permute)          (None, 113, 8, 64)        0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 113, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 113, 128)         

In [28]:
model.layers[4].output_shape

(None, 113, 8, 64)

Encoding the training and testing data to One-Hot Encoded form.

In [0]:
from sklearn.preprocessing import OneHotEncoder
def prepare_targets(y_train, y_test):
	ohe = OneHotEncoder()
	ohe.fit(y_train)
	y_train_enc = ohe.transform(y_train)
	y_test_enc = ohe.transform(y_test)
	return y_train_enc.A, y_test_enc.A

Removing NULL class values.

In [30]:
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

count = 0
idx = []
for i in range(0,y_train.shape[0]):
  if(y_train[i] == 0):
    count += 1
    idx.append(i)
print(len(idx))
y_train_new = np.delete(y_train,idx)
y_train_new = y_train_new.reshape(-1,1)
print(y_train_new.shape)

print(X_train.shape)
X_train_new = np.delete(X_train,idx,axis=0)
print(X_train_new.shape)

count = 0
idx = []
for i in range(0,y_test.shape[0]):
  if(y_test[i] == 0):
    count += 1
    idx.append(i)
print(len(idx))
y_test_new = np.delete(y_test,idx,axis = 0)
print(y_test_new.shape)

X_test_new = np.delete(X_test,idx,axis = 0)
print(X_test_new.shape)

y_train_enc, y_test_enc = prepare_targets(y_train_new, y_test_new)
print(y_train_enc.shape)
print(y_test_enc.shape)


32348
(14147, 1)
(46495, 24, 113, 1)
(14147, 24, 113, 1)
8237
(1657, 1)
(1657, 24, 113, 1)
(14147, 17)
(1657, 17)


Compiling Model

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=[tf.keras.metrics.CategoricalAccuracy()])

Training Model

In [0]:
#model.fit(X_train_new, y_train_enc, batch_size=BATCH_SIZE, epochs=500,validation_split=0.1)

#Model is already trained and loaded for testing directly.

Loading Model.

In [0]:
from tensorflow.keras.models import load_model

new_model4 = load_model('task_b_without_null_working.h5')

Testing our Model on test data.

In [0]:
y_prob = new_model4.predict(X_test_new) 
#print(y_prob)

y_classes = y_prob.argmax(axis=-1)
for i in range(0,y_classes.shape[0]):
  y_classes[i] += 1



Printing F1 Score and relevent Result Metrics.

In [35]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1_score, supp = precision_recall_fscore_support(y_classes, y_test_new, average='micro')

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_classes, y_test_new))
#print(metrics)
print("PRECISION: {0}" .format(precision))
print("RECALL: {0}".format(recall))
print("F1_SCORE : {0}".format(f1_score))

[[ 50   0   3   0   0   0   0   0   0   0   0   0   0   0   0   4   0]
 [  0  72   0   2   1   0   0   0   0   0   0   0   0   0   0   4   0]
 [  8   2  57   1   0   0   0   0   0   0   0   0   0   0   1   6   0]
 [  0  21   0  80   0   0   0   0   1   0   0   0   0   0   2   2   0]
 [  0   0   0   0 199  12  10   2   0   0   2   1   1   0   4   5   0]
 [  0   0   0   0   2 125   1   1   0   0   0   0   1   0   0   7   0]
 [  0   0   0   0   9   4  82   6   2   0   1   1   0   0   0  13   0]
 [  0   0   0   0  14  15   3  65   0   1   2   0   1   0   1   6   0]
 [  0   0   0   0   0   2   1   0  32   5   5   3   1   0   0   4   0]
 [  0   0   0   0   2   2   0   0   2  34   0   1   0   0   0   4   0]
 [  0   0   0   0   1   0   1   2   1   0  25   4   0   0   1   1   0]
 [  0   0   0   0   0   0   0   1   0   1   3  15   2   2   0   0   0]
 [  0   0   0   0   0   0   2   0   1   0   2   0  60  11   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   1   1  48   0   0   0]
 [  0 